In [71]:
import cv2
import numpy as np
import os
import pandas as pd
from glob import glob
from tqdm import tqdm

In [86]:
dirname = "One-Pot_Chicken_Fajita_Pasta"
img_folder = f"/home/users/swetha/projects/personal/hand-segmentation/data/test/{dirname}"
mask_folder = f"/home/users/swetha/projects/personal/hand-segmentation/data/test/{dirname}_maskv4"
csv_folder = "/home/users/swetha/projects/personal/action-classifier/One-Pot_Chicken_Fajita_Pasta"

In [87]:
images = glob(f"{img_folder}/*.jpg")
csv_files = glob(f"{csv_folder}/*.csv")

In [88]:
csv_files

['/home/users/swetha/projects/personal/action-classifier/One-Pot_Chicken_Fajita_Pasta/resnet101_full_bs8One-Pot_Chicken_Fajita_Pasta.csv',
 '/home/users/swetha/projects/personal/action-classifier/One-Pot_Chicken_Fajita_Pasta/resnet101_full_bs8_more_augOne-Pot_Chicken_Fajita_Pasta.csv',
 '/home/users/swetha/projects/personal/action-classifier/One-Pot_Chicken_Fajita_Pasta/resnet101_full_bs8overfit_One-Pot_Chicken_Fajita_Pasta.csv',
 '/home/users/swetha/projects/personal/action-classifier/One-Pot_Chicken_Fajita_Pasta/resnet101_full_bs8_more_augoverfit_One-Pot_Chicken_Fajita_Pasta.csv']

In [89]:
df0 = pd.read_csv(csv_files[0], index_col=0)
df1 = pd.read_csv(csv_files[1], index_col=0)
df2 = pd.read_csv(csv_files[2], index_col=0)
df3 = pd.read_csv(csv_files[3], index_col=0)
result_dict = {}

In [90]:
for i in tqdm(range(1363)):
    ind = f"frame{str(i)}"
    values = [df0.loc[ind, "prediction"], df1.loc[ind, "prediction"], df2.loc[ind, "prediction"],
         df3.loc[ind, "prediction"]]
    result_dict[ind] = max(values)
    np_arr = np.load(os.path.join(mask_folder, f"{ind}_mask.npy"))
    if np_arr.sum() == 0:
        result_dict[ind] = 0

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1363/1363 [00:04<00:00, 326.92it/s]


In [96]:
for i in tqdm(reversed(range(1363))):
    if i == 1:
        break
    ind = f'frame{str(i)}'
    values = [result_dict[f'frame{str(i)}'], result_dict[f'frame{str(i - 1)}'], result_dict[f'frame{str(i - 2)}']]
    if 1 in values and 2 in values:
        result_dict[ind] = result_dict[f'frame{str(i + 1)}']
    elif values.count(1) >= 2:
        result_dict[ind] = 1
    elif values.count(2) >= 2:
        result_dict[ind] = 2
    elif values.count(0) >= 2:
        result_dict[ind] = 0

1361it [00:00, 499514.15it/s]


In [97]:
for key, value in result_dict.items():
    print(key, value)

frame0 0
frame1 0
frame2 0
frame3 0
frame4 1
frame5 1
frame6 1
frame7 1
frame8 1
frame9 1
frame10 1
frame11 1
frame12 1
frame13 1
frame14 1
frame15 1
frame16 1
frame17 1
frame18 1
frame19 1
frame20 0
frame21 0
frame22 0
frame23 0
frame24 0
frame25 0
frame26 0
frame27 0
frame28 0
frame29 0
frame30 0
frame31 0
frame32 1
frame33 1
frame34 0
frame35 0
frame36 0
frame37 0
frame38 1
frame39 1
frame40 1
frame41 1
frame42 0
frame43 0
frame44 0
frame45 0
frame46 0
frame47 0
frame48 0
frame49 0
frame50 0
frame51 0
frame52 0
frame53 0
frame54 0
frame55 0
frame56 0
frame57 0
frame58 0
frame59 0
frame60 0
frame61 1
frame62 1
frame63 1
frame64 0
frame65 0
frame66 1
frame67 1
frame68 1
frame69 1
frame70 0
frame71 0
frame72 0
frame73 2
frame74 2
frame75 2
frame76 2
frame77 2
frame78 2
frame79 2
frame80 0
frame81 0
frame82 0
frame83 0
frame84 0
frame85 0
frame86 0
frame87 0
frame88 0
frame89 0
frame90 0
frame91 0
frame92 0
frame93 0
frame94 0
frame95 2
frame96 2
frame97 2
frame98 2
frame99 2
frame100 2

In [98]:
boundaries = f"/home/users/swetha/projects/personal/action-classifier/data/{dirname}_overlay"
os.makedirs(boundaries, exist_ok=True)


def overlay_text(path):
    
    # Reading an image in default mode
    image = cv2.imread(path)

    # font
    font = cv2.FONT_HERSHEY_SIMPLEX
  
    # org
    org = (50, 50)
  
    # fontScale
    fontScale = 1
   
    # Blue color in BGR
    color = (255, 0, 0)
  
    # Line thickness of 2 px
    thickness = 2
    name = os.path.basename(path)[:-len('.jpg')]
    
    if result_dict[name] == 1:
        text = "Adding Ingredients"
    elif result_dict[name] == 2:
        text = "Stir frying"
    else:
        text = ""
   
    # Using cv2.putText() method
    image = cv2.putText(image, text, org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)
    
    cv2.imwrite(os.path.join(boundaries, name + "_overlay.png"), 
                image)


for jpg_path in tqdm(images):
    overlay_text(jpg_path)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1363/1363 [01:24<00:00, 16.22it/s]


In [99]:
images = glob(f"{img_folder}/*.jpg")
#masks = glob(f"{mask_folder}/*.npy")
img = cv2.imread(images[0])
w, h = img.shape[1], img.shape[0]


fps = 24
output_file = f'/home/users/swetha/projects/personal/action-classifier/data/{dirname}_action_output_v2.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_file, fourcc, fps, (w, h))

In [100]:
frames = sorted(glob(f"data/test/{dirname}/*.jpg"))
length = len(frames)
for i in tqdm(range(length)):
    frame_path = os.path.join(boundaries, f"frame{str(i)}_overlay.png")
    frame = cv2.imread(frame_path)
    video_writer.write(frame)
video_writer.release()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1363/1363 [00:43<00:00, 31.05it/s]


In [56]:
for key, value in result_dict.items():
    print(key, value)

frame0 0
frame1 0
frame2 0
frame3 1
frame4 1
frame5 1
frame6 1
frame7 1
frame8 1
frame9 1
frame10 1
frame11 1
frame12 1
frame13 1
frame14 1
frame15 1
frame16 1
frame17 1
frame18 0
frame19 1
frame20 0
frame21 0
frame22 0
frame23 0
frame24 0
frame25 0
frame26 0
frame27 0
frame28 1
frame29 0
frame30 0
frame31 1
frame32 1
frame33 0
frame34 0
frame35 0
frame36 0
frame37 1
frame38 1
frame39 1
frame40 1
frame41 0
frame42 0
frame43 0
frame44 0
frame45 0
frame46 0
frame47 0
frame48 0
frame49 0
frame50 0
frame51 0
frame52 0
frame53 0
frame54 0
frame55 0
frame56 0
frame57 1
frame58 0
frame59 0
frame60 1
frame61 1
frame62 1
frame63 0
frame64 0
frame65 1
frame66 1
frame67 1
frame68 1
frame69 0
frame70 0
frame71 1
frame72 0
frame73 2
frame74 2
frame75 2
frame76 0
frame77 2
frame78 2
frame79 2
frame80 1
frame81 2
frame82 0
frame83 1
frame84 2
frame85 0
frame86 0
frame87 0
frame88 0
frame89 0
frame90 0
frame91 0
frame92 0
frame93 0
frame94 2
frame95 1
frame96 2
frame97 1
frame98 2
frame99 2
frame100 0